# Markov Decision Process, Optimal Solutions, Monte Carlo Methods

# Markov Decision Process

![w=55%,h=center](diagram.pdf)

A _Markov decision process_ (MDP) is a quadruple $(𝓢, 𝓐, p, γ)$,
where:
- $𝓢$ is a set of states,

- $𝓐$ is a set of actions,

- $p(S_{t+1} = s', R_{t+1} = r | S_t = s, A_t = a)$ is a probability that
  action $a ∈ 𝓐$ will lead from state $s ∈ 𝓢$ to $s' ∈ 𝓢$, producing a _reward_ $r ∈ ℝ$,

- $γ ∈ [0, 1]$ is a _discount factor_.


Let a _return_ $G_t$ be $G_t ≝ ∑_{k=0}^∞ γ^k R_{t + 1 + k}$. The goal is to optimize $𝔼[G_0]$.


# Multi-armed Bandits as MDP

To formulate $n$-armed bandits problem as MDP, we do not need states.
Therefore, we could formulate it as:
- one-element set of states, $𝓢=\{S\}$;

- an action for every arm, $𝓐=\{a_1, a_2, …, a_n\}$;

- assuming every arm produces rewards with a distribution of $𝓝(μ_i, σ_i^2)$,
  the MDP dynamics function $p$ is defined as
  $$p(S, r | S, a_i) = 𝓝(r | μ_i, σ_i^2).$$


One possibility to introduce states in multi-armed bandits problem is to have
separate reward distribution for every state. Such generalization is
usually called _Contextualized Bandits_ problem.
Assuming that state transitions are independent on rewards and given by
a distribution $\textit{next}(s)$, the MDP dynamics function for contextualized
bandits problem is given by
$$p(s', r | s, a_i) = 𝓝(r | μ_{i,s}, σ_{i,s}^2) ⋅ \textit{next}(s'|s).$$

---
# (State-)Value and Action-Value Functions

A _policy_ $π$ computes a distribution of actions in a given state, i.e.,
$π(a | s)$ corresponds to a probability of performing an action $a$ in state
$s$.


To evaluate a quality of a policy, we define _value function_ $v_π(s)$, or
_state-value function_, as
$$v_π(s) ≝ 𝔼_π\left[G_t \middle| S_t = s\right] = 𝔼_π\left[∑\nolimits_{k=0}^∞ γ^k R_{t+k+1} \middle| S_t=s\right].$$


An _action-value function_ for a policy $π$ is defined analogously as
$$q_π(s, a) ≝ 𝔼_π\left[G_t \middle| S_t = s, A_t = a\right] = 𝔼_π\left[∑\nolimits_{k=0}^∞ γ^k R_{t+k+1} \middle| S_t=s, A_t = a\right].$$


Evidently,
$$\begin{aligned}
  v_π(s) &= 𝔼_π[q_π(s, a)], \\
  q_π(s, a) &= 𝔼_π[R_{t+1} + γv_π(S_{t+1}) | S_t = s, A_t = a].
\end{aligned}$$

# Optimal Value Functions

Optimal state-value function is defined as
$$v_*(s) ≝ \max_π v_π(s),$$
analogously
$$q_*(s, a) ≝ \max_π q_π(s, a).$$


Any policy $π_*$ with $v_{π_*} = v_*$ is called an _optimal policy_. Such policy
can be defined as $π_*(s) ≝ \argmax_a q_*(s, a) = \argmax_a 𝔼[R_{t+1} + γv_*(S_{t+1}) | S_t = s, A_t = a]$.


## Existence
Under some mild assumptions, there always exists a unique optimal state-value function,
unique optimal action-value function, and (not necessarily unique) optimal policy.
The mild assumptions are that either termination is guaranteed from all
reachable states, or $γ < 1$.


section: Dynamic Programming
# Dynamic Programming

Dynamic programming is an approach devised by Richard Bellman in 1950s.


To apply it to MDP, we now consider finite-horizon problems (i.e., with episodes
of bounded length) with finite number of states $𝓢$ and actions $𝓐$, and known
MDP dynamics $p$.


The following recursion (which must hold for an optimal value function in a MDP,
because future decisions does not depend on the current one) is usually called
the _Bellman equation_:
$$\begin{aligned}
  v_*(s) &= \max_a 𝔼\left[R_{t+1} + γ v_*(S_{t+1}) \middle| S_t=s, A_t=a\right] \\
         &= \max_a ∑_{s', r} p(s', r | s, a) \left[r + γ v_*(s')\right].
\end{aligned}$$


It can be also shown that if a value function satisfies the Bellman equation, it is
already optimal.

---
# Dynamic Programming

To turn the Bellman equation into an algorithm, we change the equal signs to assignments:
$$\begin{aligned}
v_0(s) &← 0 \\
v_{k+1}(s) &← \max_a 𝔼\left[R_{t+1} + γ v_k(S_{t+1}) \middle| S_t=s, A_t=a\right].
\end{aligned}$$


It is easy to show that if the problem consists of episodes of length at most
$T$ steps, the optimal value function is reached after $T$ iteration of the
above assignment (we can show by induction that $v_k(s)$ is the maximum return
reachable from state $s$ in $k$ steps).

---
# Relations to Graph Algorithms

Searching for optimal value functions of deterministic problems is in fact
search for the shortest path in a suitable graph.


![w=80%,mh=80%,h=center,v=middle](trellis.svg)

---
# Bellman-Ford-Moore Algorithm

$$v_{k+1}(s) ← \max_a 𝔼\left[R_{t+1} + γ v_k(S_{t+1}) \middle| S_t=s, A_t=a\right].$$

Bellman-Ford-Moore algorithm:
```python
# input: graph `g`, initial vertex `s`
for v in g.vertices: d[v] = 0 if v == s else +∞

for i in range(len(g.vertices) - 1):
  for e in g.edges:
    if d[e.source] + e.length < d[e.target]:
      d[e.target] = d[e.source] + e.length

```

---
section: Value Iteration
# Bellman Backup Operator

Our goal is now to handle also infinite horizon tasks, using discount factor of
$γ < 1$.


For any value function $v∈ℝ^{|𝓢|}$ we define _Bellman backup operator_ $B : ℝ^{|𝓢|} → ℝ^{|𝓢|}$ as
$$Bv(s) ≝ \max_a 𝔼\left[R_{t+1} + γ v(S_{t+1}) \middle| S_t=s, A_t=a\right].$$


It is not difficult to show that Bellman backup operator is a _contraction_:
$$\max_s \left|Bv_1(s) - Bv_2(s)\right| ≤ γ \max_s \left|v_1(s) - v_2(s)\right|.$$


Considering a normed vector space $ℝ^{|𝓢|}$ with sup-norm $||⋅||_∞$,
from Banach fixed-point theorem it follows there exist a _unique value function_
$v_*$ such that
$$Bv_* = v_*.$$


Such unique $v_*$ is the _optimal value function_, because it satistifes the
Bellman equation.

---
# Bellman Backup Operator

Furthermore, iterative application of $B$ on arbitrary $v$ converges to $v_*$,
because
$$||Bv - v_*||_∞ = ||Bv - Bv_*||_∞ ≤ γ||v - v_*||,$$
and therefore $B^nv → v_*$.

---
# Value Iteration Algorithm

We can turn the iterative application of Bellman backup operator into an
algorithm.
$$Bv(s) ≝ \max_a 𝔼\left[R_{t+1} + γ v(S_{t+1}) \middle| S_t=s, A_t=a\right]$$

![w=75%,h=center](value_iteration.pdf)

---
# Value Iteration Algorithm

Although we have described the so-called _synchronous_ implementation requiring
two arrays for $v$ and $Bv$, usual implementations are _asynchronous_ and modify
the value function in place (if a fixed ordering is used, usually such value
iteration is called _Gauss-Seidel_).


Even with such asynchronous update value iteration can be proven to converge,
and usually performs better in practise.

# Bellman Backup Operator as a Contraction

To show that Bellman backup operator is a contraction, we proceed as follows:
$$\begin{aligned}
||Bv_1 - Bv_2||_∞ &= ||\max_a 𝔼\left[R_{t+1} + γ v_1(S_{t+1})\right] - \max_a 𝔼\left[R_{t+1} + γ v_2(S_{t+1})\right]||_∞ \\
                  &≤ \max_a\left( || 𝔼\left[R_{t+1} + γ v_1(S_{t+1})\right] - 𝔼\left[R_{t+1} + γ v_2(S_{t+1})\right]||_∞\right) \\
                  &= \max_a\left( \left|\left| ∑\nolimits_{s', r} p\left(s', r \middle| s, a\right)γ(v_1(s') - v_2(s'))\right|\right|_∞\right) \\
                  &= γ \max_a\left(\left|\left| ∑\nolimits_{s', r} p\left(s' \middle| s, a\right)(v_1(s') - v_2(s'))\right|\right|_∞\right) \\
                  &≤ γ ||v_1 - v_2||_∞,
\end{aligned}$$

where the second line follows from $|\max_x f(x) - \max_x g(x)| ≤ \max_x |f(x) - g(x)|$
and the last line from the fact that from any given $s$ and $a$, the
$∑_{s'} p(s' | s, a)$ sums to 1.

---
# Speed of Convergence

Assuming maximum reward is $R_\textrm{max}$, we have that
$$v_*(s) ≤ ∑_{t=0}^∞ γ^t R_\textrm{max} = \frac{R_\textrm{max}}{1-γ}.$$


Starting with $v(s) ← 0$, we have
$$||B^k v - v_*||_∞ ≤ γ^k ||v - v_*||_∞ = γ^k \frac{R_\textrm{max}}{1-γ}.$$


Compare to finite horizon case, where $B^T v = v_*$.

---
section: Policy Iteration
# Policy Iteration Algorithm

We now propose another approach of computing optimal policy. The approach,
called _policy iteration_, consists of repeatedly performing policy
_evaluation_ and policy _improvement_.

## Policy Evaluation

Given a policy $π$, policy evaluation computes $v_π$.

Recall that
$$\begin{aligned}
  v_π(s) &≝ 𝔼_π\left[G_t \middle| S_t = s\right] \\
         &= 𝔼_π\left[R_{t+1} + γ v_π(S_{t+1}) \middle | S_t = s\right] \\
         &= ∑\nolimits_a π(a|s) ∑\nolimits_{s', r} p(s', r | s, a) \left[r + γ v_π(s')\right].
\end{aligned}$$

If the dynamics of the MDP $p$ is known, the above is a system of linear
equations, and therefore, $v_π$ can be computed exactly.

---
# Policy Evaluation
The equation
$$v_π(s) = ∑\nolimits_a π(a|s) ∑\nolimits_{s', r} p(s', r | s, a) \left[r + γ v_π(s')\right]$$
is called _Bellman equation for $v_π$_ and analogously to Bellman optimality
equation, it can be proven that
- under the same assumptions as before ($γ<1$ or termination), $v_π$ exists and
  is unique;
- $v_π$ is a fixed point of the Bellman equation
  $$v_{k+1}(s) = ∑\nolimits_a π(a|s) ∑\nolimits_{s', r} p(s', r | s, a) \left[r + γ v_k(s')\right];$$
- iterative application of the Bellman equation to any $v$ converges to $v_π$.

---
class: middle
# Policy Evaluation

![w=100%](policy_evaluation.pdf)

---
# Policy Improvement

Given $π$ and computed $v_π$, we would like to _improve_ the policy.
A straightforward way to do so is to define a policy using a _greedy_ action
$$\begin{aligned}
  π'(s) &≝ \argmax_a q_π(s, a) \\
        &= \argmax_a ∑\nolimits_{s', r} p(s', r | s, a) \left[r + γ v_π(s')\right].
\end{aligned}$$

For such $π'$, we can easily show that
$$q_π(s, π'(s)) ≥ v_π(s).$$

---
# Policy Improvement Theorem

Let $π$ and $π'$ be any pair of deterministic policies, such that
$q_π(s, π'(s)) ≥ v_π(s)$.

Then for all states $s$, $v_{π'}(s) ≥ v_π(s)$.


The proof is straightforward, we repeatedly expand $q_π$ and use the
assumption of the policy improvement theorem:

$$\begin{aligned}
v_π(s) &≤ q_π(s, π'(s)) \\
       &= 𝔼[R_{t+1} + γ v_π(S_{t+1}) | S_t = s, A_t = π'(s)] \\
       &= 𝔼_{π'}[R_{t+1} + γ v_π(S_{t+1}) | S_t = s] \\
       &≤ 𝔼_{π'}[R_{t+1} + γ q_π(S_{t+1}, π'(S_{t+1})) | S_t = s] \\
       &= 𝔼_{π'}[R_{t+1} + γ E[R_{t+2} + γ v_π(S_{t+2}) | S_{t+1}, A_{t+1} = π'(S_{t+1})] | S_t = s] \\
       &= 𝔼_{π'}[R_{t+1} + γ R_{t+2} + γ^2 v_π(S_{t+2}) | S_t = s] \\
       &… \\
       &≤ 𝔼_{π'}[R_{t+1} + γ R_{t+2} + γ^2 R_{t+3} + … | S_t = s] = v_{π'}(s)
\end{aligned}$$

---
# Policy Improvement Example

![w=50%](gridworld_4x4.pdf)![w=60%,mw=50%,h=center](gridworld_4x4_policy_evaluation.pdf)

---
# Policy Iteration Algorithm

Policy iteration consists of repeatedly performing policy evaluation and policy
improvement:
$$π_0 \stackrel{E}{\longrightarrow} v_{π_0} \stackrel{I}{\longrightarrow}
  π_1 \stackrel{E}{\longrightarrow} v_{π_1} \stackrel{I}{\longrightarrow}
  π_2 \stackrel{E}{\longrightarrow} v_{π_2} \stackrel{I}{\longrightarrow}
  … \stackrel{I}{\longrightarrow} π_* \stackrel{E}{\longrightarrow} v_{π_*}.$$


The result is a sequence of monotonically improving policies $π_i$. Note that
when $π' = π$, also $v_{π'} = v_π$, which means Bellman optimality equation is
fulfilled and both $v_π$ and $π$ are optimal.


Considering that there is only a finite number of policies, the optimal policy
and optimal value function can be computed in finite time (contrary to value
iteration, where the convergence is only asymptotic).


Note that when evaluating policy $π_{k+1}$, we usually start with $v_{π_k}$,
which is assumed to be a good approximation to $v_{π_{k+1}}$.

---
# Policy Iteration Algorithm
![w=70%,h=center](policy_iteration.pdf)

---
# Value Iteration as Policy Iteration

Note that value iteration is in fact a policy iteration, where policy evaluation
is performed only for one step:

$$\begin{aligned}
  π'(s) &= \argmax_a ∑\nolimits_{s', r} p(s', r | s, a) \left[r + γ v(s')\right] &\textit{(policy improvement)} \\
  v'(s) &= ∑\nolimits_a π'(a|s) ∑\nolimits_{s', r} p(s', r | s, a) \left[r + γ v(s')\right] &\textit{(one step of policy evaluation)}
\end{aligned}$$

Substituting the former into the latter, we get
$$v'(s) = \max_a ∑\nolimits_{s', r} p(s', r | s, a) \left[r + γ v(s)\right] = Bv(s).$$

---
# Generalized Policy Iteration

Therefore, it seems that to achieve convergence, it is not necessary to perform
policy evaluation exactly.

_Generalized Policy Evaluation_ is a general idea of interleaving policy
evaluation and policy improvement at various granularity.


![w=30%,mw=50%,h=center](gpi.pdf)![w=80%,mw=50%,h=center](gpi_convergence.pdf)

If both processes stabilize, we know we have obtained optimal policy.


section: Monte Carlo Methods
# Monte Carlo Methods

We now present the first algorithm for computing optimal policies without assuming
a knowledge of the environment dynamics.

However, we still assume there are finitely many states $𝓢$ and we will store
estimates for each of them.


Monte Carlo methods are based on estimating returns from complete episodes.
Furthermore, if the model (of the environment) is not known, we need to
estimate returns for the action-value function $q$ instead of $v$.


We can formulate Monte Carlo methods in the generalized policy improvement
framework.


Keeping estimated returns for the action-value function, we perform policy
evaluation by sampling one episode according to current policy. We then update
the action-value function by averaging over the observed returns, including
the sampled episode.

# Monte Carlo Methods

To guarantee convergence, we need to visit each state infinitely many times.
One of the simplest way to achieve that is to assume _exploring starts_, where
we randomly select the first state and first action, each pair with nonzero
probability.


Furthermore, if a state-action pair appears multiple times in one episode, the
sampled returns are not independent. The literature distinguishes two cases:
- _first visit_: only the first occurence of a state-action pair in an episode is
  considered
- _every visit_: all occurences of a state-action pair are considered.

Even though first-visit is easier to analyze, it can be proven that for both
approaches, policy evaluation converges. Contrary to the Reinforcement Learning:
An Introduction book, which presents first-visit algorithms, we use every-visit.

---
# Monte Carlo with Exploring Starts

![w=90%,h=center](monte_carlo_exploring_starts.pdf)


---
# Monte Carlo and $ε$-soft Policies

A policy is called $ε$-soft, if
$$π(a|s) ≥ \frac{ε}{|𝓐(s)|}.$$


For $ε$-soft policy, Monte Carlo policy evaluation also converges, without the need
of exploring starts.


We call a policy $ε$-greedy, if one action has maximum probability of
$1-ε+\frac{ε}{|A(s)|}$.


The policy improvement theorem can be proved also for the class of $ε$-soft
policies, and using<br>$ε$-greedy policy in policy improvement step, policy
iteration has the same convergence properties. (We can embed the $ε$-soft behaviour
“inside” the environment and prove equivalence.)

---
# Monte Carlo for $ε$-soft Policies

### On-policy every-visit Monte Carlo for $ε$-soft Policies
Algorithm parameter: small $ε>0$

Initialize $Q(s, a) ∈ ℝ$ arbitrarily (usually to 0), for all $s ∈ 𝓢, a ∈ 𝓐$<br>
Initialize $C(s, a) ∈ ℤ$ to 0, for all $s ∈ 𝓢, a ∈ 𝓐$

Repeat forever (for each episode):
- Generate an episode $S_0, A_0, R_1, …, S_{T-1}, A_{T-1}, R_T$,
  by generating actions as follows:
  - With probability $ε$, generate a random uniform action
  - Otherwise, set $A_t ≝ \argmax\nolimits_a Q(S_t, a)$
- $G ← 0$
- For each $t=T-1, T-2, …, 0$:
  - $G ← γG + R_{T+1}$
  - $C(S_t, A_t) ← C(S_t, A_t) + 1$
  - $Q(S_t, A_t) ← Q(S_t, A_t) + \frac{1}{C(S_t, A_t)}(G - Q(S_t, A_t))$
